In [75]:
from datetime import datetime
import os
from pathlib import Path
from typing import Tuple

from arcgis.features import FeatureLayer
from arcgis.gis import GIS
from arcgis.geoenrichment import Country
from dotenv import find_dotenv, load_dotenv
import numpy as np
import pandas as pd
from sodapy import Socrata
from sklearn.linear_model import LinearRegression

In [2]:
# using python-dotenv to load credentials from .env file as environment variables
load_dotenv(find_dotenv())

# variables for finding resources
mcpp_url = 'https://gisdata.seattle.gov/server/rest/services/SPD/SPD/MapServer/4'
seattle_socrata_url = 'data.seattle.gov'
crime_socrata_id = 'tazs-3rd5'

# useful paths
dir_prj = Path.cwd().parent
dir_data = dir_prj/'data'
dir_int = dir_data/'interim'

# path to where crime data is cached
crime_df_pth = dir_int/'crime_df.pickle'

## Download Data From Seattle Open Data

Seattle uses Socrata and ArcGIS Open Data to provide open data resources. The crime data is only available through Socrata, so we can use the `Socrata` object from the `sodapy` package to retrieve the data as a list of JSON objects. From there we can convert these JSON objects to a Pandas data frame for subsequent analysis.

In [3]:
# get today's date to work backwards from
today = datetime.today()

start_year = today.year - 1  # start one year ago
start_month = today.month - 1  # one month back since data typically is available for last month

# example of how to format string - used below to create SoQL query string for Socrata request
f'{start_year}-{start_month:02d}-01'

'2021-05-01'

In [4]:
# if the cached data does not exist
if not crime_df_pth.exists():

    # create a connection to Seattle Socrata
    socrata = Socrata(seattle_socrata_url, 
                      app_token=os.getenv('SEATTLE_SOCRATA_APP_TOKEN'), 
                      username=os.getenv('SEATTLE_SOCRATA_API_KEY'), 
                      password=os.getenv('SEATTLE_SOCRATA_SECRET_KEY')
                     )

    # get all the data - since requires pagnation, this only returns a generator to get all the data
    crime_gen = socrata.get_all(crime_socrata_id, 
                            where=f"offense_start_datetime >= '{start_year}-{start_month:02d}-01'"
                           )

    # create a pandas data frame by loading from the get_all - forces the generator to execute
    crime_df = pd.DataFrame.from_dict(crime_gen)
    
    # convert the start date time to a real datetime and set as the index
    crime_df['offense_start_datetime'] = pd.to_datetime(crime_df['offense_start_datetime'])
    crime_df.set_index('offense_start_datetime', inplace=True)
    
    # cache the results for later runs
    crime_df.to_pickle(crime_df_pth)

# if the cached data already exists, just load it
else:
    crime_df = pd.read_pickle(crime_df_pth)
    
# make sure data is for time period we are interested in since I've sometimes cached data from way further back
crime_df = crime_df.loc[crime_df.index >= (today - pd.DateOffset(years=1, months=1))]

# ensure no leading or trailing spaces in the mcpp column
crime_df.mcpp = crime_df.mcpp.str.strip()

# fix typo encountered in data
crime_df.mcpp = crime_df.mcpp.str.replace('CAPTIOL', 'CAPITOL')
    
crime_df.info()
crime_df.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 80408 entries, 2021-05-02 07:00:00 to 2022-05-31 17:57:00
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   report_number           80408 non-null  object
 1   offense_id              80408 non-null  object
 2   offense_end_datetime    58042 non-null  object
 3   report_datetime         80408 non-null  object
 4   group_a_b               80408 non-null  object
 5   crime_against_category  80408 non-null  object
 6   offense_parent_group    80408 non-null  object
 7   offense                 80408 non-null  object
 8   offense_code            80408 non-null  object
 9   precinct                80405 non-null  object
 10  sector                  80408 non-null  object
 11  beat                    80408 non-null  object
 12  mcpp                    80408 non-null  object
 13  _100_block_address      74305 non-null  object
 14  longitude          

,report_number,offense_id,offense_end_datetime,report_datetime,group_a_b,crime_against_category,offense_parent_group,offense,offense_code,precinct,sector,beat,mcpp,_100_block_address,longitude,latitude
offense_start_datetime,,,,,,,,,,,,,,,,
2021-05-02 07:00:00,2021-108318,23602074746,2021-05-02T20:00:00.000,2021-05-04T06:04:30.000,A,PROPERTY,MOTOR VEHICLE THEFT,Motor Vehicle Theft,240,S,O,O2,GEORGETOWN,1ST AVE S / S ORCAS ST,-122.334216046,47.551903260
2021-05-02 07:15:00,2021-106702,23535159984,2021-05-02T07:36:00.000,2021-05-02T09:13:50.000,A,PROPERTY,BURGLARY/BREAKING&ENTERING,Burglary/Breaking & Entering,220,E,E,E2,CAPITOL HILL,18XX BLOCK OF BOYLSTON AVE,-122.323447280,47.617653300
2021-05-02 07:19:00,2021-106691,23535110645,NaN,2021-05-02T09:16:28.000,A,PROPERTY,LARCENY-THEFT,Shoplifting,23C,E,C,C3,CENTRAL AREA/SQUIRE PARK,22XX BLOCK OF E MADISON ST,-122.303098800,47.618795460
2021-05-02 07:33:00,2021-106699,23542485285,2021-05-02T07:33:00.000,2021-05-02T11:04:56.000,A,PROPERTY,DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY,Destruction/Damage/Vandalism of Property,290,S,R,R3,COLUMBIA CITY,36XX BLOCK OF S EDMUNDS ST,-122.286913708,47.558659550
2021-05-02 07:33:00,2021-106699,23558879881,2021-05-02T07:33:00.000,2021-05-02T11:04:56.000,A,PROPERTY,ROBBERY,Robbery,120,S,R,R3,COLUMBIA CITY,36XX BLOCK OF S EDMUNDS ST,-122.286913708,47.558659550


In [90]:
crime_mcpp_dt_srs = crime_df.groupby(['mcpp', pd.Grouper(freq='W')]).size()
crime_mcpp_dt_srs.name = 'crime_count'
crime_mcpp_dt_df = crime_mcpp_dt_srs.reset_index()

# remove unknown
crime_mcpp_dt_df = crime_mcpp_dt_df.loc[crime_mcpp_dt_df.mcpp != 'UNKNOWN']

crime_mcpp_dt_df.info()
crime_mcpp_dt_df.sample(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3262 entries, 0 to 3319
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   mcpp                    3262 non-null   object        
 1   offense_start_datetime  3262 non-null   datetime64[ns]
 2   crime_count             3262 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 101.9+ KB


,mcpp,offense_start_datetime,crime_count
1989,MORGAN,2021-12-26,5
294,BITTERLAKE,2021-06-20,22
454,CAPITOL HILL,2022-04-24,80
1448,JUDKINS PARK/NORTH BEACON HILL,2021-06-27,13
3192,UNIVERSITY,2022-03-20,51


In [8]:
mcpp_lyr = FeatureLayer(mcpp_url)

mcpp_lyr

<FeatureLayer url:"https://gisdata.seattle.gov/server/rest/services/SPD/SPD/MapServer/4">

In [33]:
mcpp_df = mcpp_lyr.query(out_fields='NEIGHBORHOOD', out_sr=4326).sdf.drop(columns='OBJECTID')
mcpp_df.columns = ['mcpp', 'SHAPE']
mcpp_df.spatial.set_geometry('SHAPE')
assert mcpp_df.spatial.validate()

mcpp_df.info()
mcpp_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   mcpp    58 non-null     string  
 1   SHAPE   58 non-null     geometry
dtypes: geometry(1), string(1)
memory usage: 1.0 KB


,mcpp,SHAPE
0,ALASKA JUNCTION,"{""rings"": [[[-122.37346169215971, 47.569868845..."
1,ALKI,"{""rings"": [[[-122.37241408619865, 47.588688879..."
2,BALLARD NORTH,"{""rings"": [[[-122.36607680181163, 47.705160617..."
3,BALLARD SOUTH,"{""rings"": [[[-122.36602339866104, 47.675985346..."
4,BELLTOWN,"{""rings"": [[[-122.3513600695938, 47.6185863995..."


In [10]:
gis = GIS(os.getenv('ESRI_GIS_URL'), username=os.getenv('ESRI_GIS_USERNAME'), password=os.getenv('ESRI_GIS_PASSWORD'))

gis

GIS @ https://baqa.mapsqa.arcgis.com version:10.1

In [11]:
usa = Country('usa', gis=gis)

usa

<Country - United States (GIS @ https://baqa.mapsqa.arcgis.com version:10.1)>

In [36]:
ev_df = usa.enrich_variables[
    (usa.enrich_variables.data_collection.str.lower().str.contains('key'))  # "key" data collection
    & (usa.enrich_variables.name.str.lower().str.endswith('cy'))  # current year variables
].drop_duplicates('name').reset_index(drop=True)  # ensure no duplicates (possible if pulling from multiple data collections)

# set the index to the name in lowercase
ev_df.index = ev_df.name.str.lower()

ev_df

,name,alias,data_collection,enrich_name,enrich_field_name,description,vintage,units
name,,,,,,,,
totpop_cy,TOTPOP_CY,2021 Total Population,KeyUSFacts,KeyUSFacts.TOTPOP_CY,KeyUSFacts_TOTPOP_CY,2021 Total Population (Esri),2021,count
gqpop_cy,GQPOP_CY,2021 Group Quarters Population,KeyUSFacts,KeyUSFacts.GQPOP_CY,KeyUSFacts_GQPOP_CY,2021 Group Quarters Population (Esri),2021,count
divindx_cy,DIVINDX_CY,2021 Diversity Index,KeyUSFacts,KeyUSFacts.DIVINDX_CY,KeyUSFacts_DIVINDX_CY,2021 Diversity Index (Esri),2021,count
tothh_cy,TOTHH_CY,2021 Total Households,KeyUSFacts,KeyUSFacts.TOTHH_CY,KeyUSFacts_TOTHH_CY,2021 Total Households (Esri),2021,count
avghhsz_cy,AVGHHSZ_CY,2021 Average Household Size,KeyUSFacts,KeyUSFacts.AVGHHSZ_CY,KeyUSFacts_AVGHHSZ_CY,2021 Average Household Size (Esri),2021,count
medhinc_cy,MEDHINC_CY,2021 Median Household Income,KeyUSFacts,KeyUSFacts.MEDHINC_CY,KeyUSFacts_MEDHINC_CY,2021 Median Household Income (Esri),2021,currency
avghinc_cy,AVGHINC_CY,2021 Average Household Income,KeyUSFacts,KeyUSFacts.AVGHINC_CY,KeyUSFacts_AVGHINC_CY,2021 Average Household Income (Esri),2021,currency
pci_cy,PCI_CY,2021 Per Capita Income,KeyUSFacts,KeyUSFacts.PCI_CY,KeyUSFacts_PCI_CY,2021 Per Capita Income (Esri),2021,currency
tothu_cy,TOTHU_CY,2021 Total Housing Units,KeyUSFacts,KeyUSFacts.TOTHU_CY,KeyUSFacts_TOTHU_CY,2021 Total Housing Units (Esri),2021,count


In [49]:
enrich_df = usa.enrich(mcpp_df, enrich_variables=ev_df, return_geometry=False)

enrich_df.info()
enrich_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 26 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   mcpp                               58 non-null     string 
 1   source_country                     58 non-null     object 
 2   aggregation_method                 58 non-null     object 
 3   population_to_polygon_size_rating  58 non-null     float64
 4   apportionment_confidence           58 non-null     float64
 5   has_data                           58 non-null     int64  
 6   totpop_cy                          58 non-null     int64  
 7   gqpop_cy                           58 non-null     int64  
 8   divindx_cy                         58 non-null     float64
 9   tothh_cy                           58 non-null     int64  
 10  avghhsz_cy                         58 non-null     float64
 11  medhinc_cy                         58 non-null     int64  
 

,mcpp,source_country,aggregation_method,population_to_polygon_size_rating,apportionment_confidence,has_data,totpop_cy,gqpop_cy,divindx_cy,tothh_cy,...,renter_cy,vacant_cy,medval_cy,avgval_cy,popgrw10_cy,hhgrw10_cy,famgrw10_cy,dpop_cy,dpopwrk_cy,dpopres_cy
0,ALASKA JUNCTION,USA,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,16369,290,44.3,8411,...,4646,386,714205,769011,3.03,3.38,3.24,15647,9161,6486
1,ALKI,USA,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,7109,15,37.3,3796,...,1619,328,963208,1098265,0.91,0.78,0.97,5648,3045,2603
2,BALLARD NORTH,USA,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,29728,253,38.1,12652,...,3430,340,797390,863355,1.07,0.89,1.07,25037,13236,11801
3,BALLARD SOUTH,USA,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,24683,226,42.0,13604,...,8778,914,787224,862277,2.59,2.70,2.67,27042,18956,8086
4,BELLTOWN,USA,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,11652,458,62.2,8197,...,6148,1100,784631,906881,2.74,2.92,3.16,20200,16581,3619


In [60]:
id_col = 'mcpp'

keep_cols = [c for c in enrich_df.columns if (c == id_col) or (c in ev_df.index)]

keep_cols

['mcpp',
 'totpop_cy',
 'gqpop_cy',
 'divindx_cy',
 'tothh_cy',
 'avghhsz_cy',
 'medhinc_cy',
 'avghinc_cy',
 'pci_cy',
 'tothu_cy',
 'owner_cy',
 'renter_cy',
 'vacant_cy',
 'medval_cy',
 'avgval_cy',
 'dpop_cy',
 'dpopwrk_cy',
 'dpopres_cy']

In [79]:
demographics_df = enrich_df.loc[:,keep_cols].set_index(id_col, drop=True)

demographics_df.info()
demographics_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 58 entries, ALASKA JUNCTION to INTERNATIONAL DISTRICT
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   totpop_cy   58 non-null     int64  
 1   gqpop_cy    58 non-null     int64  
 2   divindx_cy  58 non-null     float64
 3   tothh_cy    58 non-null     int64  
 4   avghhsz_cy  58 non-null     float64
 5   medhinc_cy  58 non-null     int64  
 6   avghinc_cy  58 non-null     int64  
 7   pci_cy      58 non-null     int64  
 8   tothu_cy    58 non-null     int64  
 9   owner_cy    58 non-null     int64  
 10  renter_cy   58 non-null     int64  
 11  vacant_cy   58 non-null     int64  
 12  medval_cy   58 non-null     int64  
 13  avgval_cy   58 non-null     int64  
 14  dpop_cy     58 non-null     int64  
 15  dpopwrk_cy  58 non-null     int64  
 16  dpopres_cy  58 non-null     int64  
dtypes: float64(2), int64(15)
memory usage: 8.2 KB


,totpop_cy,gqpop_cy,divindx_cy,tothh_cy,avghhsz_cy,medhinc_cy,avghinc_cy,pci_cy,tothu_cy,owner_cy,renter_cy,vacant_cy,medval_cy,avgval_cy,dpop_cy,dpopwrk_cy,dpopres_cy
mcpp,,,,,,,,,,,,,,,,,
ALASKA JUNCTION,16369,290,44.3,8411,1.91,107440,140261,72403,8797,3765,4646,386,714205,769011,15647,9161,6486
ALKI,7109,15,37.3,3796,1.87,153758,189606,99208,4124,2177,1619,328,963208,1098265,5648,3045,2603
BALLARD NORTH,29728,253,38.1,12652,2.33,129879,177574,75402,12992,9222,3430,340,797390,863355,25037,13236,11801
BALLARD SOUTH,24683,226,42.0,13604,1.80,102152,146090,80706,14518,4826,8778,914,787224,862277,27042,18956,8086
BELLTOWN,11652,458,62.2,8197,1.37,104765,148608,104679,9297,2049,6148,1100,784631,906881,20200,16581,3619


In [93]:
crime_demog_df = crime_mcpp_dt_df.join(demographics_df, on=id_col)

crime_demog_df.info()
crime_demog_df.sample(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3262 entries, 0 to 3319
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   mcpp                    3262 non-null   object        
 1   offense_start_datetime  3262 non-null   datetime64[ns]
 2   crime_count             3262 non-null   int64         
 3   totpop_cy               3147 non-null   float64       
 4   gqpop_cy                3147 non-null   float64       
 5   divindx_cy              3147 non-null   float64       
 6   tothh_cy                3147 non-null   float64       
 7   avghhsz_cy              3147 non-null   float64       
 8   medhinc_cy              3147 non-null   float64       
 9   avghinc_cy              3147 non-null   float64       
 10  pci_cy                  3147 non-null   float64       
 11  tothu_cy                3147 non-null   float64       
 12  owner_cy                3147 non-null   float64 

,mcpp,offense_start_datetime,crime_count,totpop_cy,gqpop_cy,divindx_cy,tothh_cy,avghhsz_cy,medhinc_cy,avghinc_cy,pci_cy,tothu_cy,owner_cy,renter_cy,vacant_cy,medval_cy,avgval_cy,dpop_cy,dpopwrk_cy,dpopres_cy
3051,SOUTH DELRIDGE,2021-09-26,4,1650.0,0.0,88.4,611.0,2.70,55297.0,78475.0,29551.0,648.0,275.0,336.0,37.0,485119.0,512091.0,1361.0,652.0,709.0
2354,PHINNEY RIDGE,2021-05-09,10,10225.0,217.0,35.2,4638.0,2.16,127255.0,178113.0,80828.0,4835.0,2885.0,1754.0,197.0,858199.0,908466.0,9496.0,5956.0,3540.0
1043,FREMONT,2021-05-30,34,14107.0,169.0,44.4,7877.0,1.77,109873.0,154936.0,83889.0,8236.0,2550.0,5328.0,359.0,788215.0,826971.0,15599.0,12094.0,3505.0
1352,HIGHLAND PARK,2021-10-31,12,10750.0,36.0,78.1,4132.0,2.59,67816.0,96953.0,37324.0,4855.0,2637.0,1495.0,723.0,478492.0,532108.0,8479.0,3614.0,4865.0
1235,GREENWOOD,2021-10-10,24,18812.0,63.0,58.2,8784.0,2.13,100490.0,128966.0,60361.0,9179.0,4522.0,4262.0,395.0,627871.0,662022.0,16417.0,9655.0,6762.0


In [94]:
crime_demog_df['crime_per_capita'] = crime_demog_df['crime_count'] / crime_demog_df['totpop_cy']

crime_demog_df.crime_per_capita = crime_demog_df.crime_per_capita.replace(np.inf, 0.0)

crime_demog_df.sort_values('crime_per_capita', ascending=False)[['mcpp', 'offense_start_datetime', 'crime_count', 'totpop_cy', 'dpop_cy', 'crime_per_capita']]

,mcpp,offense_start_datetime,crime_count,totpop_cy,dpop_cy,crime_per_capita
694,COMMERCIAL DUWAMISH,2021-05-16,6,2.0,1060.0,3.0
695,COMMERCIAL DUWAMISH,2021-05-23,5,2.0,1060.0,2.5
737,COMMERCIAL DUWAMISH,2022-05-22,4,2.0,1060.0,2.0
725,COMMERCIAL DUWAMISH,2022-01-30,4,2.0,1060.0,2.0
714,COMMERCIAL DUWAMISH,2021-10-31,4,2.0,1060.0,2.0
...,...,...,...,...,...,...
1720,MADRONA/LESCHI,2022-05-01,15,NaN,NaN,NaN
1721,MADRONA/LESCHI,2022-05-08,12,NaN,NaN,NaN
1722,MADRONA/LESCHI,2022-05-15,18,NaN,NaN,NaN
1723,MADRONA/LESCHI,2022-05-22,16,NaN,NaN,NaN


In [95]:
crime_demog_df['crime_per_capita'] = crime_demog_df['crime_count'] / crime_demog_df['dpop_cy']

crime_demog_df.crime_per_capita = crime_demog_df.crime_per_capita.replace(np.inf, 0.0)

crime_demog_df.sort_values('crime_per_capita', ascending=False)[['mcpp', 'offense_start_datetime', 'crime_count', 'totpop_cy', 'dpop_cy', 'crime_per_capita']]

,mcpp,offense_start_datetime,crime_count,totpop_cy,dpop_cy,crime_per_capita
3078,SOUTH DELRIDGE,2022-04-03,16,1650.0,1361.0,0.011756
3069,SOUTH DELRIDGE,2022-01-30,14,1650.0,1361.0,0.010287
821,EASTLAKE - EAST,2021-05-16,9,1109.0,946.0,0.009514
1151,GENESEE,2022-05-22,14,1656.0,1552.0,0.009021
826,EASTLAKE - EAST,2021-07-04,8,1109.0,946.0,0.008457
...,...,...,...,...,...,...
1720,MADRONA/LESCHI,2022-05-01,15,NaN,NaN,NaN
1721,MADRONA/LESCHI,2022-05-08,12,NaN,NaN,NaN
1722,MADRONA/LESCHI,2022-05-15,18,NaN,NaN,NaN
1723,MADRONA/LESCHI,2022-05-22,16,NaN,NaN,NaN


In [96]:
def get_slope_coef(unique_value:str, y_col:str='crime_per_capita', df:pd.DataFrame=crime_demog_df, dt_col:str='offense_start_datetime', id_col:str='mcpp')->Tuple:
    
    # ensure input columns are available
    assert unique_value in df[id_col].values
    assert y_col in df.columns
    assert dt_col in df.columns
    assert id_col in df.columns

    # filter to just the data for the input mcpp
    mcpp_subset_df = df[df.mcpp == unique_value]

    # extract and format the X and y inputs for modeling
    X = mcpp_subset_df[dt_col].astype(int).values.reshape(-1, 1)
    y = mcpp_subset_df[y_col].values

    # use linear regression from SciKit Learn to get the slope coefficient
    lm = LinearRegression()
    lm.fit(X, y)
    slope_coef = lm.coef_[0]

    return (unique_value, slope_coef)


get_slope_coef('ALASKA JUNCTION')

('ALASKA JUNCTION', 1.3249140323247507e-20)

In [97]:
coef_df = pd.DataFrame([get_slope_coef(mcpp) for mcpp in crime_mcpp_dt_df.mcpp.unique()], columns=['mcpp', 'slope_coef'])

coef_df.info()
coef_df.head()

ValueError: Input y contains NaN.